In [ ]:
import tensorflow as tf
from tensorflow import keras
import cv2 as cv
import numpy as np
import pickle
import warnings; warnings.filterwarnings('ignore')

# %sudo docker run -it -p 8888:8888 -p 6006:6006 tensorflow/tensorflow:nightly-py3-jupyter

(x_train, y_train), (x_val, y_val) = keras.datasets.fashion_mnist.load_data()

In [72]:
x_train
y_train

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [12]:
def preprocess(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)

    return x, y

def create_dataset(xs, ys, n_classes=10):
    ys = tf.one_hot(ys, depth=n_classes)
    return tf.data.Dataset.from_tensor_slices((xs, ys)) \
        .map(preprocess) \
        .shuffle(len(ys)) \
        .batch(128)

In [13]:
train_dataset = create_dataset(x_train, y_train)
val_dataset = create_dataset(x_val, y_val)

In [14]:
model = keras.Sequential([
    keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=192, activation='relu'),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dense(units=10, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', 
              loss=tf.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
    train_dataset.repeat(), 
    epochs=30, 
    steps_per_epoch=500,
    validation_data=val_dataset.repeat(), 
    validation_steps=2
)

with open(b"model.pickle", "wb") as file:
    pickle.dump(model, file)

In [34]:
model_load = pickle.load(open(b"model.pickle", "rb"))

In [ ]:
predictions = model.predict(val_dataset)

In [ ]:
# import os; os.environ['TENSORBOARD_BINARY'] = '/home/hanchau/proj/ds/skl/bin/tensorboard'
# %reload_ext tensorboard
# %tensorboard --logdir logs/fit

